In [56]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import BatchNormalization

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

EPOCHS = 1000

In [57]:
df = pd.read_csv('us_merged.csv', sep=",")

In [58]:
df

,Unnamed: 0,title,review,rating
0,0,Creepy and atmospheric,The Hollywood version of 'Dark Water' is based...,8/10
1,1,Teenagers!,"Don't know about you all, but I've sort of had...",9/10
2,2,Pretty good with a great performance by Connely,"Dark Water is another one of those slow PG-13,...",8/10
3,3,Dark Water Has Depth,Of all the recent remakes of Japanese horror f...,9/10
4,4,Good character-driven human drama; I blame the...,"Once more, a great injustice has been done by ...",7/10
...,...,...,...,...
3297,214,Uninspired,"I haven't seen that many scary movies, but one...",7/10
3298,215,Great progress for further Asian remakes! The ...,I came back from the 'The Uninvited' and I am ...,8/10
3299,216,The ending makes it filled with flaws and conf...,I found The Uninvited on IMDb and saw it got a...,6/10
3300,217,"A knock off of ""The Other"" not half bad but","A not bad, nifty thriller that unfortunately b...",5/10


In [59]:
df = df.drop('Unnamed: 0', axis=1)

In [60]:
df

,title,review,rating
0,Creepy and atmospheric,The Hollywood version of 'Dark Water' is based...,8/10
1,Teenagers!,"Don't know about you all, but I've sort of had...",9/10
2,Pretty good with a great performance by Connely,"Dark Water is another one of those slow PG-13,...",8/10
3,Dark Water Has Depth,Of all the recent remakes of Japanese horror f...,9/10
4,Good character-driven human drama; I blame the...,"Once more, a great injustice has been done by ...",7/10
...,...,...,...
3297,Uninspired,"I haven't seen that many scary movies, but one...",7/10
3298,Great progress for further Asian remakes! The ...,I came back from the 'The Uninvited' and I am ...,8/10
3299,The ending makes it filled with flaws and conf...,I found The Uninvited on IMDb and saw it got a...,6/10
3300,"A knock off of ""The Other"" not half bad but","A not bad, nifty thriller that unfortunately b...",5/10


In [61]:
df['review'][88]

'Oh boy.First of all I\'d like to say I\'m not against remakes in general. The remake has a very big disadvantage since it will doubtlessly be compared to the original (which has to be somewhat good, otherwise he wouldn\'t inspire the remake) so it\'s not really fair to be too hard on it. Besides, if you\'d loved the original it\'s always fun to look how other director/cast interpreted the thing.What I DO hate is when movie people start "improving" the original, putting in all sorts of their ideas and solutions in order to 1) distance themselves from the original and 2) make the movie more suited for intended audience. Perhaps some directors can pull this off, but usually (and especially when it is the case of American remakes) the "improving" consists of heavy overexplaining of the plot and augmented use of special effects. I don\'t get this - does Hollywood really think American audience is soo stupid they must be led by hand the entire movie, and God forbid something ambiguous was l

In [62]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from tqdm import tqdm
from nltk import sent_tokenize,word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [63]:
main = []

# Storing all punctuations using RE library like !;,"% etc
re_puncs = re.compile('[%s]' % re.escape(string.punctuation))
# Storing all stop words like a, an, the, when, there, this etc
stop_word  = set(stopwords.words('english'))
stop_word.add("im")
# print(stop_word)


In [64]:
sentences_listed = [line.split(" ") for line in df['review']]

import gensim

word_model = gensim.models.Word2Vec(sentences = sentences_listed, vector_size = 100)
words = list(word_model.wv.key_to_index)
print(len(words))

8729


In [65]:
!pip install gensim

In [66]:
word_model.wv.most_similar("actress")

[('role', 0.930316686630249),
 ('actress,', 0.9255497455596924),
 ('performance', 0.9204145669937134),
 ('actor', 0.893725574016571),
 ('plays', 0.8878854513168335),
 ('Connelly', 0.8864920735359192),
 ('lead,', 0.8788108825683594),
 ('actress.', 0.878707230091095),
 ('Emily', 0.8781460523605347),
 ('Dorfman', 0.8764450550079346)]

In [67]:
word_model.wv.most_similar("death")

[('spirit', 0.9410275220870972),
 ('house', 0.9261094927787781),
 ('mother', 0.9227659702301025),
 ('body', 0.9194740056991577),
 ('woman', 0.918242335319519),
 ('cursed', 0.9178704619407654),
 ('Dahlia', 0.9173790812492371),
 ('husband', 0.915257453918457),
 ('living', 0.9152496457099915),
 ('daughter', 0.9146366119384766)]

In [68]:
word_model.wv.most_similar("scary")

[('bad', 0.909307599067688),
 ('good', 0.8006821274757385),
 ('scary,', 0.7488455176353455),
 ('truly', 0.7389134764671326),
 ('scary.', 0.7292941808700562),
 ('quite', 0.7193005084991455),
 ('Not', 0.7153157591819763),
 ('bad,', 0.7132468819618225),
 ('all,', 0.7007725238800049),
 ('really', 0.6968408823013306)]

In [69]:
# Making Lemmatizing object
lem = WordNetLemmatizer()
# Using Porter Stemmer
p_stem = PorterStemmer()

# Traversing whole dataset
for i in tqdm(range(len(df['review']))):
    # Tokenization
    tokens = word_tokenize(str(df['review'][i]))
    # Converting all characters to lower case
    tokens = [w.lower() for w in tokens]
    # Remove all punctuations from sentenses
    tokens = [re_puncs.sub('', w) for w in tokens]
    # Checking all words is alphabets or not
    tokens = [i for i in tokens if i.isalpha()]
    # Removing all stop words from the sentenses
    tokens = [w for w in tokens if w not in stop_word]
    # Doing Lemmatizing of words
    tokens = [lem.lemmatize(w) for w in tokens]
    # Stemming process
    tokens = [p_stem.stem(w) for w in tokens]
    # Finally convert to string
    r = ' '.join(tokens)
    # Storing the final string into main list
    main.append(r)


100%|██████████| 3302/3302 [00:31<00:00, 106.20it/s]


In [70]:
df['translated'] = main

In [71]:
df

,title,review,rating,translated
0,Creepy and atmospheric,The Hollywood version of 'Dark Water' is based...,8/10,hollywood version dark water base japanes horr...
1,Teenagers!,"Don't know about you all, but I've sort of had...",9/10,nt know sort teenag walter sall dark water flo...
2,Pretty good with a great performance by Connely,"Dark Water is another one of those slow PG-13,...",8/10,dark water anoth one slow atmospher creepi gho...
3,Dark Water Has Depth,Of all the recent remakes of Japanese horror f...,9/10,recent remak japanes horror film say dark wate...
4,Good character-driven human drama; I blame the...,"Once more, a great injustice has been done by ...",7/10,great injustic done imdb voter dark water curr...
...,...,...,...,...
3297,Uninspired,"I haven't seen that many scary movies, but one...",7/10,nt seen mani scari movi one thing know sure mo...
3298,Great progress for further Asian remakes! The ...,I came back from the 'The Uninvited' and I am ...,8/10,came back uninvit surpris horror fanat everi h...
3299,The ending makes it filled with flaws and conf...,I found The Uninvited on IMDb and saw it got a...,6/10,found uninvit imdb saw got okay rate horrorsus...
3300,"A knock off of ""The Other"" not half bad but","A not bad, nifty thriller that unfortunately b...",5/10,bad nifti thriller unfortun break rule ca nt t...


In [72]:
#df['translated'][88]

In [73]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['translated'])

In [74]:
print(len(tokenizer.word_counts))

17869


In [75]:
counter = 0
for key, value in tokenizer.word_counts.items():
    if value > 2400:
        print(key, value)
        counter+=1

print(counter)

horror 3659
film 6132
nt 5906
ring 2443
good 2610
one 3182
watch 2558
like 3149
movi 9678
see 2647
10


**grupa 1** scary/scared, afraid, fright(ened), alarmed, shocked, horror/horrified, threatening, hair-rising, horrendous

In [76]:
count1 = 0
num=tokenizer.word_counts['scari']
print(num)
count1+=num

1958


In [77]:
num=tokenizer.word_counts['scare']
print(num)
count1+=num

1763


In [78]:
num=tokenizer.word_counts['afraid']
print(num)
count1+=num

85


In [79]:
num=tokenizer.word_counts['fright']
print(num)
count1+=num

81


In [80]:
num=tokenizer.word_counts['frighten']
print(num)
count1+=num

308


In [81]:
num=tokenizer.word_counts['alarm']
print(num)
count1+=num

8


In [82]:
num=tokenizer.word_counts['shock']
print(num)
count1+=num

263


In [83]:
num=tokenizer.word_counts['horrifi']
print(num)
count1+=num

94


In [84]:
num=tokenizer.word_counts['horrif']
print(num)
count1+=num

57


In [85]:
num=tokenizer.word_counts['threaten']
print(num)
count1+=num

26


In [86]:
num=tokenizer.word_counts['horrend']
print(num)
count1+=num

12


In [88]:
print('WYNIK DLA GRUPY PIERWSZEJ TOOOOO',count1)

WYNIK DLA GRUPY PIERWSZEJ TOOOOO 4674


**grupa 2** fear, petrified, terror/terrified, distress, helpless(ness), anxiety/anxious, worried, apprehension, creepy, uneasy, disturbing, chilling, eerie, spooky(?), intimidating, overwhelming, unnerving

In [89]:
count2 = 0
num=tokenizer.word_counts['fear']
print(num)
count2+=num

280


In [90]:
num=tokenizer.word_counts['petrifi']
print(num)
count2+=num

7


In [91]:
num=tokenizer.word_counts['terror']
print(num)
count2+=num

153


In [92]:
num=tokenizer.word_counts['terrifi']
print(num)
count2+=num

230


In [93]:
num=tokenizer.word_counts['distress']
print(num)
count2+=num

13


In [94]:
num=tokenizer.word_counts['helpless']
print(num)
count2+=num

9


In [95]:
num=tokenizer.word_counts['anxious']
print(num)
count2+=num

2


In [96]:
num=tokenizer.word_counts['worri']
print(num)
count2+=num

43


In [97]:
num=tokenizer.word_counts['creepy']
print(num)
count2+=num

2


In [98]:
num=tokenizer.word_counts['creepi']
print(num)
count2+=num

950


In [99]:
num=tokenizer.word_counts['uneasi']
print(num)
count2+=num

24


In [100]:
num=tokenizer.word_counts['disturb']
print(num)
count2+=num

326


In [101]:
num=tokenizer.word_counts['chilli']
print(num)
count2+=num

4


In [102]:
num=tokenizer.word_counts['eeri']
print(num)
count2+=num

159


In [103]:
num=tokenizer.word_counts['overwhelm']
print(num)
count2+=num

9


In [104]:
num=tokenizer.word_counts['overwhelming']
print(num)
count2+=num

1


In [105]:
print('WYNIK DLA GRUPY DRUGIEJ TOOOOO',count2)

WYNIK DLA GRUPY DRUGIEJ TOOOOO 2212


**SKOJARZONKA**

In [110]:
word_model.wv.most_similar("death")

[('spirit', 0.9410275220870972),
 ('house', 0.9261094927787781),
 ('mother', 0.9227659702301025),
 ('body', 0.9194740056991577),
 ('woman', 0.918242335319519),
 ('cursed', 0.9178704619407654),
 ('Dahlia', 0.9173790812492371),
 ('husband', 0.915257453918457),
 ('living', 0.9152496457099915),
 ('daughter', 0.9146366119384766)]

In [112]:
word_model.wv.most_similar("ghost")

[('evil', 0.8381465077400208),
 ('quest', 0.8083985447883606),
 ('chunk', 0.8079286217689514),
 ('killer', 0.7935214042663574),
 ('haunting', 0.784801185131073),
 ('spirit', 0.782246470451355),
 ('videotape,', 0.7810903191566467),
 ('girl.', 0.780897319316864),
 ('mystery', 0.7798375487327576),
 ('haunted', 0.7791330218315125)]

In [113]:
word_model.wv.most_similar("horror")

[('Horror', 0.7911938428878784),
 ('slasher', 0.7207719683647156),
 ('type', 0.6950723528862),
 ('fan', 0.6817806363105774),
 ('PG-13', 0.6753058433532715),
 ('Hollywood', 0.6732533574104309),
 ('foreign', 0.6661958694458008),
 ('Asian', 0.641019880771637),
 ('scariest', 0.6406490802764893),
 ('scary', 0.6337063312530518)]

In [114]:
word_model.wv.most_similar("kill")

[('die', 0.930810272693634),
 ('call', 0.9239935278892517),
 ('stop', 0.9234817028045654),
 ('live', 0.9173609018325806),
 ('show', 0.9056143164634705),
 ('save', 0.8955711722373962),
 ('die.', 0.8884414434432983),
 ('decide', 0.888350248336792),
 ('help', 0.8856567144393921),
 ('haunt', 0.8838581442832947)]

In [115]:
word_model.wv.most_similar("place")

[('curse', 0.8987290263175964),
 ('world', 0.8953240513801575),
 ('living', 0.8880655169487),
 ('virus', 0.8846330046653748),
 ('jumping', 0.8803708553314209),
 ('room', 0.8782447576522827),
 ('tv', 0.8755472302436829),
 ('move', 0.8730922341346741),
 ('TV', 0.8724016547203064),
 ('suddenly', 0.8717635869979858)]

In [116]:
word_model.wv.most_similar("body")

[('walking', 0.9641663432121277),
 ('voice', 0.9612115025520325),
 ('suddenly', 0.9542981386184692),
 ('red', 0.9541471600532532),
 ('white', 0.9535545110702515),
 ('apartment,', 0.9535278081893921),
 ('custody', 0.9487176537513733),
 ('hospital', 0.9485521912574768),
 ("Samara's", 0.9468812346458435),
 ('water', 0.9448438286781311)]

In [119]:
word_model.wv.most_similar("ending")

[('script', 0.808991551399231),
 ('plot', 0.7762845158576965),
 ('story', 0.7393538355827332),
 ('scary,', 0.7373859286308289),
 ('lame.', 0.7356363534927368),
 ('storyline', 0.7326868176460266),
 ('bad,', 0.7272818088531494),
 ('acting', 0.7260394096374512),
 ('ending.', 0.7228914499282837),
 ('scary.', 0.7219255566596985)]

In [120]:
word_model.wv.most_similar("gore")

[('blood', 0.947770357131958),
 ('cheap', 0.8859180808067322),
 ('elements,', 0.8753091096878052),
 ('violence', 0.873715877532959),
 ('shock', 0.8672831654548645),
 ('guts', 0.8563405275344849),
 ('thrills', 0.8474271297454834),
 ('purpose,', 0.8467035293579102),
 ('scares,', 0.8417893052101135),
 ('tactics', 0.837113618850708)]